<a href="https://colab.research.google.com/github/linyuehzzz/census_privacy/blob/main/franklin_abm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


####**Read data**

Building data

In [2]:
!pip install geopandas

     |████████████████████████████████| 994 kB 3.0 MB/s 
     |████████████████████████████████| 15.4 MB 82 kB/s 
     |████████████████████████████████| 6.6 MB 45.2 MB/s 


In [64]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import geopandas as gpd

filename_buildings = 'franklin/buildings/franklin_buildingsv1.json'
data_buildings = gpd.read_file(filename_buildings)
data_buildings

/content/gdrive/My Drive/Colab Notebooks/census_privacy


,OBJECTID_1,OBJECTID,Building_I,GEOID10,PUMA,Shape_Leng,land_value,near_schoo,accessibil,Area,land_val_1,land_val_2,land_val_3,near_sch_1,near_sch_2,accessib_1,area_norme,score1,score2,score3,score4,score5,score6,prob1,prob2,prob3,prob4,prob5,prob6,lat,lon,Shape_Length,Shape_Area,geometry
0,1,1,1,390490073943030,3904101,0.000779,11.371515,1.469520,0,263.381747,0.041935,0.053736,0.000000e+00,0.655379,1,0.000000,0.001184,0.005580,0.002791,0.006608,0.003851,0.002644,0.000568,0.044962,0.043204,0.048824,0.050354,0.050932,0.074844,40.034603,-82.769582,0.000779,2.779850e-08,"POLYGON ((-82.76948 40.03471, -82.76953 40.034..."
1,2,2,2,390490072031031,3904101,0.000336,11.246373,0.792078,0,66.362896,0.025874,0.039652,0.000000e+00,0.814248,1,0.000000,0.000298,0.004916,0.001442,0.005908,0.002473,0.002724,0.000143,0.004931,0.002192,0.006244,0.004050,0.009111,0.003128,40.101399,-82.771929,0.000336,7.011000e-09,"POLYGON ((-82.77188 40.10144, -82.77197 40.101..."
2,3,3,3,390490072031031,3904101,0.000267,12.244342,0.433578,0,39.905402,0.470084,0.294711,2.378500e-11,0.898321,1,0.000000,0.000179,0.026558,0.022817,0.020803,0.017073,0.002934,0.000086,0.026642,0.034673,0.021984,0.027963,0.009812,0.001880,40.111748,-82.772499,0.000267,4.216500e-09,"POLYGON ((-82.77246 40.11178, -82.77251 40.111..."
3,4,4,4,390490073943030,3904101,0.000690,11.500648,1.250005,0,219.030016,0.066589,0.072382,0.000000e+00,0.706858,1,0.000000,0.000984,0.006857,0.003853,0.007767,0.004796,0.002712,0.000472,0.055253,0.059637,0.057388,0.062714,0.052240,0.062239,40.036686,-82.772915,0.000690,2.311800e-08,"POLYGON ((-82.77279 40.03673, -82.77304 40.036..."
4,5,5,5,390490073962108,3904101,0.000548,11.920041,1.812932,8,138.618666,0.232572,0.170544,0.000000e+00,0.574845,1,0.016260,0.000623,0.018972,0.016586,0.017485,0.015116,0.005778,0.003968,0.001207,0.001112,0.001291,0.001187,0.001561,0.001293,40.004223,-82.773029,0.000548,1.462400e-08,"POLYGON ((-82.77294 40.00425, -82.77300 40.004..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390124,390125,390125,390125,390490080003007,3904111,0.000968,11.238415,2.790384,0,373.625217,0.025062,0.038873,0.000000e+00,0.345620,1,0.000000,0.001679,0.003764,0.002297,0.004762,0.003313,0.001899,0.000805,0.028677,0.036380,0.029308,0.034823,0.024999,0.031853,40.025717,-83.244110,0.000968,3.941550e-08,"POLYGON ((-83.24400 40.02583, -83.24411 40.025..."
390125,390126,390126,390126,390490080001007,3904111,0.000887,10.416450,3.159957,2,406.954095,0.000443,0.003608,0.000000e+00,0.258951,1,0.004065,0.001829,0.003539,0.002443,0.003690,0.002606,0.002610,0.001795,0.165056,0.195167,0.159922,0.182905,0.166761,0.198998,39.969960,-83.245228,0.000887,4.289700e-08,"POLYGON ((-83.24511 39.97005, -83.24524 39.970..."
390126,390127,390127,390127,390490098001032,3904111,0.000584,10.697505,3.202852,0,194.178110,0.002078,0.008752,0.000000e+00,0.248892,1,0.000000,0.000873,0.001726,0.000665,0.002112,0.001063,0.001206,0.000419,0.015849,0.010984,0.016370,0.013088,0.021029,0.019581,39.880342,-83.245832,0.000584,2.044200e-08,"POLYGON ((-83.24573 39.88030, -83.24582 39.880..."
390127,390128,390128,390128,390490098001032,3904111,0.000596,11.497328,3.362483,0,198.879741,0.065832,0.071844,0.000000e+00,0.211456,1,0.000000,0.000894,0.004645,0.003758,0.005582,0.004707,0.001098,0.000429,0.042660,0.062060,0.043273,0.057962,0.019136,0.020055,39.887494,-83.247253,0.000596,2.093900e-08,"POLYGON ((-83.24714 39.88745, -83.24725 39.887..."


Household data

In [11]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import pandas as pd

filename_census = 'franklin/microdata/franklin_householdsv1.csv'
data_census = pd.read_csv(filename_census)
data_census

/content/gdrive/My Drive/Colab Notebooks/census_privacy


,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,HouseholdID_G,HouseholdID,Income,Race,Children,Building_ID
0,45682,39,49,110,1001,390490001101001,45682,1,22500.0,9,0,343585
1,432911,39,49,110,1002,390490001101002,432911,1,36200.0,1,1,23275
2,432912,39,49,110,1002,390490001101002,432912,2,22000.0,1,1,41973
3,432913,39,49,110,1002,390490001101002,432913,3,36100.0,1,1,231411
4,432914,39,49,110,1002,390490001101002,432914,4,21800.0,1,1,231411
...,...,...,...,...,...,...,...,...,...,...,...,...
458651,282548,39,49,10700,1020,390490107001020,282548,142,47500.0,1,0,383856
458652,282549,39,49,10700,1020,390490107001020,282549,143,3500.0,1,0,256797
458653,282550,39,49,10700,1020,390490107001020,282550,144,8000.0,1,0,365487
458654,282551,39,49,10700,1020,390490107001020,282551,145,23000.0,2,0,21490


In [ ]:
data_census["Income"].describe()

count    458656.000000
mean      44204.419483
std       48514.044305
min       -6900.000000
25%       17500.000000
50%       32300.000000
75%       54333.333333
max      849000.000000
Name: Income, dtype: float64

####**Using assigned probability**


- Low income households (< 20000) with school age children
- Low income households without school age children
- Median income households (20000 - 50000) with school age children
- Median income households without school age children
- High income households (> 50000) with school age children
- High income households without school age children

In [ ]:
import random

GEOIDs = data_census["GEOID10"].unique()
data_census["Building_ID"] = 0

flag = 0
for GEOID in GEOIDs:
    # print(GEOID)
    sub_census = data_census[data_census["GEOID10"] == GEOID]
    sub_buildings = data_buildings[data_buildings["GEOID10"] == str(GEOID)]
    # print(len(sub_census), len(sub_buildings))

    if len(sub_census) > 0 and len(sub_buildings) > 0:
        sub_building_ids = sub_buildings["Building_ID"].values.tolist()
        # type 1
        cat1 = sub_census[(sub_census["Income"] < 20000) & (sub_census["Children"] == 1)]
        if len(cat1) > 0:
            ids = random.choices(sub_building_ids, weights=sub_buildings["prob1"].values.tolist(), k=len(cat1))
            data_census.loc[(data_census["GEOID10"] == GEOID) & (data_census["Income"] < 20000) & 
                            (data_census["Children"] == 1), "Building_ID"] = ids
        # type 2
        cat2 = sub_census[(sub_census["Income"] < 20000) & (sub_census["Children"] == 0)]
        if len(cat2) > 0:
            ids = random.choices(sub_building_ids, weights=sub_buildings["prob2"].values.tolist(), k=len(cat2))
            data_census.loc[(data_census["GEOID10"] == GEOID) & (data_census["Income"] < 20000) & 
                            (data_census["Children"] == 0), "Building_ID"] = ids
        # type 3
        cat3 = sub_census[(sub_census["Income"] >= 20000) & (sub_census["Income"] <= 50000) & (sub_census["Children"] == 1)]
        if len(cat3) > 0:
            ids = random.choices(sub_building_ids, weights=sub_buildings["prob3"].values.tolist(), k=len(cat3))
            data_census.loc[(data_census["GEOID10"] == GEOID) & (data_census["Income"] >= 20000) & 
                            (data_census["Income"] <= 50000) & (data_census["Children"] == 1), "Building_ID"] = ids
        # type 4
        cat4 = sub_census[(sub_census["Income"] >= 20000) & (sub_census["Income"] <= 50000) & (sub_census["Children"] == 0)]
        if len(cat4) > 0:
            ids = random.choices(sub_building_ids, weights=sub_buildings["prob4"].values.tolist(), k=len(cat4))
            data_census.loc[(data_census["GEOID10"] == GEOID) & (data_census["Income"] >= 20000) & 
                            (data_census["Income"] <= 50000) & (data_census["Children"] == 0), "Building_ID"] = ids
        # type 5
        cat5 = sub_census[(sub_census["Income"] > 50000) & (sub_census["Children"] == 1)]
        if len(cat5) > 0:
            ids = random.choices(sub_building_ids, weights=sub_buildings["prob5"].values.tolist(), k=len(cat5))
            data_census.loc[(data_census["GEOID10"] == GEOID) & (data_census["Income"] > 50000) & 
                            (data_census["Children"] == 1), "Building_ID"] = ids
        # type 6
        cat6 = sub_census[(sub_census["Income"] > 50000) & (sub_census["Children"] == 0)]
        if len(cat6) > 0:
            ids = random.choices(sub_building_ids, weights=sub_buildings["prob6"].values.tolist(), k=len(cat6))
            data_census.loc[(data_census["GEOID10"] == GEOID) & (data_census["Income"] > 50000) & 
                            (data_census["Children"] == 0), "Building_ID"] = ids
    elif len(sub_census) > 0 and len(sub_buildings) == 0:
        data_census.loc[data_census["GEOID10"] == GEOID, "Building_ID"] = "b."
        flag += 1

data_census.head()

,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,HouseholdID_G,HouseholdID,Income,Race,Children,Building_ID
0,45682,39,49,110,1001,390490001101001,45682,1,22500.0,9,0,343585
1,432911,39,49,110,1002,390490001101002,432911,1,36200.0,1,1,23275
2,432912,39,49,110,1002,390490001101002,432912,2,22000.0,1,1,41973
3,432913,39,49,110,1002,390490001101002,432913,3,36100.0,1,1,231411
4,432914,39,49,110,1002,390490001101002,432914,4,21800.0,1,1,231411


In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import geopandas as gpd

filename_census = 'franklin/microdata/franklin_householdsv1.csv'
data_census.to_csv(filename_census, index=False)

/content/gdrive/My Drive/Colab Notebooks/census_privacy


In [ ]:
len(data_census[data_census["Building_ID"] == "b."])

3103

Assign geometry.

In [9]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import geopandas as gpd

filename_block = 'franklin/buildings/franklin_block10.json'
data_block = gpd.read_file(filename_block)
data_block

/content/gdrive/My Drive/Colab Notebooks/census_privacy


,OBJECTID_1,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYPE,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Shape_Leng,PUMA,Shape_Length,Shape_Area,geometry
0,1,1,39,049,006500,4019,390490065004019,Block 4019,G5040,U,19234,U,S,98391,2149,+40.0001827,-083.0802471,0.016835,3904104,0.016835,1.060361e-05,"POLYGON ((-83.08283 40.00301, -83.08280 40.002..."
1,2,2,39,049,007812,2017,390490078122017,Block 2017,G5040,U,19234,U,S,0,2575,+40.0376070,-083.0355367,0.017067,3904105,0.017067,2.716620e-07,"POLYGON ((-83.03954 40.03505, -83.03952 40.035..."
2,3,3,39,049,008380,3046,390490083803046,Block 3046,G5040,R,,,S,0,1099,+39.8858862,-083.0260845,0.002871,3904110,0.002871,1.156665e-07,"POLYGON ((-83.02613 39.88562, -83.02617 39.885..."
3,4,4,39,049,007531,1013,390490075311013,Block 1013,G5040,U,19234,U,S,78927,1630,+40.0527883,-082.9453223,0.017112,3904106,0.017112,8.502607e-06,"POLYGON ((-82.94881 40.05366, -82.94883 40.053..."
4,5,5,39,049,007207,1025,390490072071025,Block 1025,G5040,U,19234,U,S,0,1765,+40.1094785,-082.8392880,0.008204,3904101,0.008204,1.864805e-07,"POLYGON ((-82.83938 40.10938, -82.83945 40.109..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22821,22822,22822,39,049,008230,2008,390490082302008,Block 2008,G5040,U,19234,U,S,18112,0,+39.9455080,-083.1190358,0.005576,3904110,0.005576,1.908732e-06,"POLYGON ((-83.11996 39.94567, -83.11997 39.945..."
22822,22823,22823,39,049,006430,1011,390490064301011,Block 1011,G5040,U,19234,U,S,22932,0,+40.0096213,-083.0492725,0.007361,3904104,0.007361,2.418887e-06,"POLYGON ((-83.05067 40.00971, -83.05070 40.009..."
22823,22824,22824,39,049,009326,1001,390490093261001,Block 1001,G5040,U,19234,U,S,3335,0,+39.9193678,-082.8749815,0.002875,3904108,0.002875,3.512945e-07,"POLYGON ((-82.87536 39.91920, -82.87534 39.919..."
22824,22825,22825,39,049,008242,3029,390490082423029,Block 3029,G5040,U,19234,U,S,12821,0,+39.9760232,-083.0885334,0.006785,3904110,0.006785,1.351719e-06,"POLYGON ((-83.08998 39.97569, -83.08992 39.975..."


In [ ]:
import random
from shapely.geometry import Point

data_census["lon"] = data_census["lat"] = 0

for building in data_census["Building_ID"].unique():
    if building != "b.":
        sub_buildings = data_buildings[data_buildings["Building_I"] == int(building)]
        data_census.loc[data_census["Building_ID"] == building, "lon"] = sub_buildings["lon"].values[0]
        data_census.loc[data_census["Building_ID"] == building, "lat"] = sub_buildings["lat"].values[0]
    else:
        sub_census = data_census[data_census["Building_ID"] == building]
        for GEOID in sub_census["GEOID10"].unique():
            N = len(sub_census["GEOID10"])
            polygon = data_block[data_block["GEOID10"] == str(GEOID)].geometry
            bounds = list(polygon.bounds.values)
            flag = 0
            pts = []
            while True:
                x = random.uniform(bounds[0][0], bounds[0][2])
                y = random.uniform(bounds[0][1], bounds[0][3])
                if polygon.contains(Point(x, y)).any():
                    pt = [x, y]
                    pts.append(pt)
                    flag += 1
                if flag == N:
                    break
            print(pts)
            data_census.loc[(data_census["GEOID10"] == GEOID) & (data_census["Building_ID"] == building), 
                            ["lon", "lat"]] = pts

data_census.head()